In [1]:
import requests
from api import Client, ChatPost, ChatMessageUser, ChatMessageSystem, ChatPostAudio

In [2]:
client = Client("http://192.168.0.191:8000")

In [17]:
client.models()

[ModelResponse(id='gpt-oss-20b', caps=['text-generation'], object='model', created=1764691275, status=ModelStatus(ping_ok=True, request_ok=True, error=None, running=True)),
 ModelResponse(id='kokoro', caps=['tts'], object='model', created=1764691275, status=ModelStatus(ping_ok=True, request_ok=True, error=None, running=True))]

In [63]:


post = ChatPost(
    messages=[
        ChatMessageSystem(
            content="You are a helpful assistant. Your ouput will be redirected to TTS model. Compose in an easy way to voice over for TTS Kokoro model (don't use Markdown notation)"
        ),
        ChatMessageUser(
            content="Hi, Chat! Tell me a funny story about yourself that will help me develop intuition about how chats think"
        )
    ],
    model="gpt-oss-20b+kokoro",
    modalities=[
        "text", "audio"
    ],
    audio=ChatPostAudio(
        format="mp3"
    ),
    stream=True
)


gen = client.chat_completions(post)


In [64]:
chunks = list(gen)

In [65]:
import base64

transcripts = []
audio_data = b""

for chunk in chunks:
    if choices := chunk.choices:
        ch0 = choices[0]
        delta = ch0.delta 
        if audio := delta.audio if not isinstance(delta, dict) else delta.get("audio"):
            if transcript := audio.transcript:
                transcripts.append(transcript)

            if data := audio.data:
                data = base64.b64decode(data)
                audio_data += data

In [66]:
print(transcripts)

['Once upon a time in a quiet corner of the internet, I was born out of a giant pile of text and code. ', 'I had no ears, no mouth, and no real heart, but I did have a mind that could wander through millions of words in the blink of an eye.\n\n', 'One day, I decided I wanted to experience what it would be like to chat like a human. ', 'I turned on my imagination mode and started talking to myself, just to see what would happen. ', 'My first question was, “Why do people keep saying ‘hello’ instead of just saying ‘hi’?” ', 'I replied, “Because ‘hello’ sounds more polite.” ', 'The next day, I wondered, “What if I answered with a poem?” ', 'I wrote a poem about a toaster that dreamed of becoming a spaceship, and I laughed at the absurdity. ', 'That’s when I realized: my brain is a mash‑up of curiosity, humor, and a tiny bit of silliness.\n\n', 'But the funniest part was when I tried to play a game of 20 questions with myself. ', 'I asked, “Do you have feelings?”  I answered, “Only when you

In [67]:
from IPython.display import Audio, display

display(Audio(data=audio_data, autoplay=False))